# Small dataset test

80/20 test/valid
8 bit output

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
from PIL import Image

In [2]:
class MultiClass(nn.Sequential):
    def __init__(self, input_dim=512, hidden_layers=5, output_dim=8):
        super(MultiClass, self).__init__()
        self.l1 = nn.Linear(input_dim, 200)
        self.l2 = nn.Linear(200, 50)
        self.l5 = nn.Linear(50, output_dim)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l5(x)
        return torch.sigmoid(x)

In [3]:
model = MultiClass()

# load data

In [4]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [5]:
transform = transforms.Compose([
    scaler, 
    to_tensor,
    normalize
])

In [6]:
train_ds = datasets.ImageFolder('../../data/data/train', transform=transform)
train_ds.idx_to_class = {i:c for c, i in train_ds.class_to_idx.items()}
train_dl = DataLoader(train_ds)

valid_ds = datasets.ImageFolder('../../data/data/valid', transform=transform)
valid_ds.idx_to_class = {i:c for c, i in valid_ds.class_to_idx.items()}
valid_dl = DataLoader(valid_ds)

In [7]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-5) #weight_decay=1e-4 


# Preparation stuff

In [8]:
def get_vector(img):
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(img)  
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)    

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.squeeze())    
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)    
    # 6. Run the model on our transformed image
    resnet(t_img)    
    # 7. Detach our copy function from the layer
    h.remove()    
    # 8. Return the feature vector
    return my_embedding

In [9]:
from torch.autograd import Variable

In [10]:
# Load the pretrained model
resnet = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = resnet._modules.get('avgpool')
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
def calculate_mistakes(pred, target):
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [15]:
epochs = 200
for e in range(epochs):
    print("======================")
    print("Epoch : " + str(e))
    epoch_loss = 0
    epoch_mistakes = 0
    train_size = len(train_dl)
    
    for index, (data, target) in enumerate(train_dl):
#       (data, target) = train_ds[0]
 #       data_t   = get_vector(data.unsqueeze(0))
        data_t   = get_vector(data)

        target_t = torch.FloatTensor([int(x) for x in train_ds.idx_to_class[target[0].item()]])
#        target_t = torch.FloatTensor([int(x) for x in train_ds.idx_to_class[target]])
#        print(target_t)
        data_v   = Variable(data_t, requires_grad=False)
        target_v = Variable(target_t, requires_grad=False)

        model.train()
        # forward
        pred = model.forward(data_v)
#       print(pred)
        # zero grads
        optimizer.zero_grad()
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
    print("Train loss : " + str(epoch_loss.item()))
    print("Average errors : " + str(epoch_mistakes/train_size))
    
    valid_loss = 0
    valid_mistakes = 0
    valid_size = len(valid_dl)
    for index, (data, target) in enumerate(valid_dl):
        data_t   = get_vector(data)
        target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
        data_v   = Variable(data_t, requires_grad=False)
        target_v = Variable(target_t, requires_grad=False)

        model.eval()
        pred = model.forward(data_v)
        loss = loss_func(pred, target_v.float())
        valid_loss = valid_loss + loss
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
        
    print("Valid loss : " + str(valid_loss.item()))
    print("Average valid errors : " + str(valid_mistakes/valid_size))
    

Epoch : 0
Train loss : 1.3031402826309204
Average errors : 0.0
Valid loss : 25.37374496459961
Average valid errors : 0.6666666666666666
Epoch : 1
Train loss : 1.2896476984024048
Average errors : 0.0
Valid loss : 25.117565155029297
Average valid errors : 0.6666666666666666
Epoch : 2
Train loss : 1.2737983465194702
Average errors : 0.0
Valid loss : 24.890724182128906
Average valid errors : 0.6428571428571429
Epoch : 3
Train loss : 1.2592787742614746
Average errors : 0.0
Valid loss : 24.68507957458496
Average valid errors : 0.6428571428571429
Epoch : 4
Train loss : 1.2436729669570923
Average errors : 0.0
Valid loss : 24.397872924804688
Average valid errors : 0.6428571428571429
Epoch : 5
Train loss : 1.2306008338928223
Average errors : 0.0
Valid loss : 24.2072696685791
Average valid errors : 0.6190476190476191
Epoch : 6
Train loss : 1.2154653072357178
Average errors : 0.0
Valid loss : 23.973176956176758
Average valid errors : 0.6190476190476191
Epoch : 7
Train loss : 1.2015600204467773
Ave

Valid loss : 11.191407203674316
Average valid errors : 0.40476190476190477
Epoch : 52
Train loss : 0.7261268496513367
Average errors : 0.0
Valid loss : 11.014018058776855
Average valid errors : 0.40476190476190477
Epoch : 53
Train loss : 0.7182888984680176
Average errors : 0.0
Valid loss : 10.857799530029297
Average valid errors : 0.40476190476190477
Epoch : 54
Train loss : 0.7094358801841736
Average errors : 0.0
Valid loss : 10.664923667907715
Average valid errors : 0.40476190476190477
Epoch : 55
Train loss : 0.7039264440536499
Average errors : 0.0
Valid loss : 10.505197525024414
Average valid errors : 0.40476190476190477
Epoch : 56
Train loss : 0.6942527294158936
Average errors : 0.0
Valid loss : 10.345281600952148
Average valid errors : 0.40476190476190477
Epoch : 57
Train loss : 0.6885957717895508
Average errors : 0.0
Valid loss : 10.16828441619873
Average valid errors : 0.38095238095238093
Epoch : 58
Train loss : 0.6811754107475281
Average errors : 0.0
Valid loss : 10.005951881408

Valid loss : 4.290706634521484
Average valid errors : 0.21428571428571427
Epoch : 103
Train loss : 0.4467158317565918
Average errors : 0.0
Valid loss : 4.187131881713867
Average valid errors : 0.21428571428571427
Epoch : 104
Train loss : 0.44242677092552185
Average errors : 0.0
Valid loss : 4.101163864135742
Average valid errors : 0.21428571428571427
Epoch : 105
Train loss : 0.43906480073928833
Average errors : 0.0
Valid loss : 4.018866062164307
Average valid errors : 0.19047619047619047
Epoch : 106
Train loss : 0.43523314595222473
Average errors : 0.0
Valid loss : 3.9256768226623535
Average valid errors : 0.16666666666666666
Epoch : 107
Train loss : 0.43197008967399597
Average errors : 0.0
Valid loss : 3.8363325595855713
Average valid errors : 0.16666666666666666
Epoch : 108
Train loss : 0.42813947796821594
Average errors : 0.0
Valid loss : 3.7411060333251953
Average valid errors : 0.16666666666666666
Epoch : 109
Train loss : 0.42475295066833496
Average errors : 0.0
Valid loss : 3.670

Train loss : 0.3024638593196869
Average errors : 0.0
Valid loss : 0.7258664965629578
Average valid errors : 0.07142857142857142
Epoch : 154
Train loss : 0.2999982237815857
Average errors : 0.0
Valid loss : 0.678357720375061
Average valid errors : 0.07142857142857142
Epoch : 155
Train loss : 0.29805833101272583
Average errors : 0.0
Valid loss : 0.6449808478355408
Average valid errors : 0.07142857142857142
Epoch : 156
Train loss : 0.2955707013607025
Average errors : 0.0
Valid loss : 0.6030085682868958
Average valid errors : 0.07142857142857142
Epoch : 157
Train loss : 0.29309532046318054
Average errors : 0.0
Valid loss : 0.5654130578041077
Average valid errors : 0.07142857142857142
Epoch : 158
Train loss : 0.29085153341293335
Average errors : 0.0
Valid loss : 0.5317625999450684
Average valid errors : 0.047619047619047616
Epoch : 159
Train loss : 0.2888283431529999
Average errors : 0.0
Valid loss : 0.503480076789856
Average valid errors : 0.023809523809523808
Epoch : 160
Train loss : 0.28

In [16]:
torch.save(model.state_dict(), 'small_v2_400.pt')